# Preprocessing Data for Machine Learning

**Outline:**

* Exploring and Preparing Data
  - Checking Data Types
  - Handling Missing Data
      + Dropping Mising Data
      + Imputation
* Feature Scaling (Standardizing Data)
  - Log normalization
  - Scaling
* Feature Engineering
  - Numerical Features
  - Categorical Features (Encoding)
  - Text
* Feature Selection
* Data Splitting
  - Class Distribution

_Datasets: UFO sightings, Gapminder._

In [1]:
# Import necessary modules
import pandas as pd
import re # Regular Expressions

## Exploring and Preparing Data

In [2]:
# Read in the file
df_ufo = pd.read_csv('datasets/ufo.csv')

# Print the first 3 rows
df_ufo.head(3)

,date,city,state,country,type,seconds,length_of_time,desc,recorded,lat,long
0,11/3/2011 19:21,woodville,wi,us,unknown,1209600.0,2 weeks,Red blinking objects similar to airplanes or s...,12/12/2011,44.9530556,-92.291111
1,10/3/2004 19:05,cleveland,oh,us,circle,30.0,30sec.,Many fighter jets flying towards UFO,10/27/2004,41.4994444,-81.695556
2,9/25/2009 21:00,coon rapids,mn,us,cigar,0.0,NaN,Green&#44 red&#44 and blue pulses of light tha...,12/12/2009,45.1200000,-93.287500


### Checking Data Types

In [3]:
# Print the data types of the df
print(df_ufo.dtypes)

# Print the count of each data type
print(df_ufo.dtypes.value_counts())

date               object
city               object
state              object
country            object
type               object
seconds           float64
length_of_time     object
desc               object
recorded           object
lat                object
long              float64
dtype: object

In [4]:
# Print the head of 'date' column
print(df_ufo.date.head())

# Change the date column to type datetime
df_ufo["date"] = pd.to_datetime(df_ufo["date"])

0     11/3/2011 19:21
1     10/3/2004 19:05
2     9/25/2009 21:00
3    11/21/2002 05:45
4     8/19/2010 12:55
Name: date, dtype: object


In [5]:
# Print the head of lat column.
print(df_ufo.lat.head())

0    44.9530556
1    41.4994444
2    45.1200000
3    36.0213889
4     51.083333
Name: lat, dtype: object


In [ ]:
# Convert the 'lat' column to type int
df_ufo['lat'] = df_ufo['lat'].astype(float)

The above code gives an error, since there is one row in the lat column which is not a float (e.g. '2/3/2012'). There are 9 such wrong entries in the column.

In [6]:
# Print the number of rows in "lat" column which has '/' in its string
df_ufo.lat.apply(lambda x: "/" in x).sum()

9

Additionally, some variables are actually categorical, and it is inefficient to work pandas `ojectt` type. We can use `.astype('category')` to change the data type of the variables to category.

In [ ]:
# Define the lambda function
categorize_label = lambda x: x.astype('category')

# Convert df[LABELS] to a categorical type
df.label = df[['label']].apply(categorize_label, axis=0)

# Print the converted dtypes
print(df[LABELS].dtypes)

### Handling Missing Data

#### Dropping Missing Data

In [10]:
# Drop the row with wrong lat information
df_ufo = df_ufo[df_ufo.lat.apply(lambda x: "/" not in x)]

# Convert the 'lat' column to type int
df_ufo['lat'] = df_ufo['lat'].astype(float)

# Drop the row with wrong lat information
#df_ufo.drop(df_ufo[df_ufo['lat']=='2/3/2012'].index, axis=0, inplace = True)

Another way to deal with wrong entries is encoding them as `NaN`. With this technique, we can deal with missing data easily and more efficiently. For instance, we can use `fillna()`, `dropna()`, and `Imputer()` with them. 

In [ ]:
# Convert entries in "lat" column with '/' to NaN
# df_ufo.lat[df_ufo.lat.apply(lambda x: "/" in x)] = np.nan

# Print the number of NaNs
# print(df_ufo.lat.isnull().sum())

# Drop missing values and print shape of new DataFrame
# df_ufo = df_ufo.dropna()

In [11]:
# Print the info for for the df_ufo
df_ufo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4926 entries, 0 to 4934
Data columns (total 11 columns):
date              4926 non-null datetime64[ns]
city              4926 non-null object
state             4512 non-null object
country           4255 non-null object
type              4776 non-null object
seconds           4926 non-null float64
length_of_time    4785 non-null object
desc              4926 non-null object
recorded          4926 non-null object
lat               4926 non-null float64
long              4926 non-null float64
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 461.8+ KB


In [12]:
# Take a sample of the country column and check whether it has missing value
df_ufo.country.isnull().sample(5)

2833    False
443     False
3562    False
3736    False
986     False
Name: country, dtype: bool

In [13]:
# Number of missing values in all columns
df_ufo.isnull().sum()

date                0
city                0
state             414
country           671
type              150
seconds             0
length_of_time    141
desc                0
recorded            0
lat                 0
long                0
dtype: int64

In [ ]:
# Print the number of missing values in the 'city' column
print(df_ufo.city.isnull().sum())

# print the shape of the df
print(df_ufo.shape)

# Subset the dataset with missing values in the 'city' column and print the shape of the new df
print(df_ufo[df_ufo.city.isnull()].shape)

# Subset the dataset with non-missing values in the 'city' column and print the shape of the new df
print(df_ufo[df_ufo.city.notnull()].shape)

In [ ]:
# Keep only rows where length_of_time, state, and type are not null
df_ufo = df_ufo[df_ufo.length_of_time.notnull() & 
          df_ufo.state.notnull() & 
          df_ufo.type.notnull() &
          df_ufo.country.notnull()]

# Print out the shape of the new dataset
df_ufo.shape

In [ ]:
# Number of missing values in all columns
df_ufo.isnull().sum()

We can also use the `dropna()` function to remove data, with parameters `axis=0` for rows and `thresh=` for the desired threshold. For instance, when threshold is 8, we'll drop the columns/features which have less than 8 non-missing values (or at least 4 missing values).

In [ ]:
# Drop rows with at least 4 missing values
df_ufo.dropna(axis=0, thresh=8).shape

#### Imputation

> `fit()` method learns model parameters (e.g. mean and standard deviation for normalization) from a training set, and `transform()` method applies this transformation model to unseen data.

In [ ]:
# Import the Imputer module
from sklearn.preprocessing import Imputer

# Setup the Imputation transformer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

# Fit
imp.fit(X)

# Transform
X = imp.transform(X)

### Gapminder Dataset

EDA for the Gapminder dataset is done in another notebook. We'll just import the dataset, print some info and continue with the following section.

In [ ]:
# Read the gapminder file into df_gm
df_gm = pd.read_csv("datasets/gapminder.csv")

df_gm.info()

There are no missing values in the Gapminder dataset. all the variable types are float except for "Region". It is a categorical variable and will be discussed later.

In [ ]:
df_gm.head(3)

## Feature Scaling

![Big and Small](https://eazybi.com/static/img/blog_page/posts/2015_12_14/small_vs_big.jpg "Big and Small")

We may need to standardize the range of independent variables (or features) in order to increase predicton accuracy, especially when the range of values of different features varies significantly. Standardization is required for certain Machine Learning algorithms (such as K-nearest neighbors and SVM) to work properly. 
(Note: Wine dataset is used in this section)

$$ z = {x - \mu \over \sigma} $$

In [ ]:
df_ufo.var()

In [ ]:
df_gm.var()

### Log Normalization

In [ ]:
# Print out the variance of the Proline column
print(wine.Proline.var())

# Apply the log normalization function to the Proline column
wine['Proline_log'] = np.log(wine.Proline)

# Check the variance of the Proline column again
print(wine.Proline.var())

### Scaling Data - Standardizing Columns

In [ ]:
# Import StandardScaler from scikit-learn
from sklearn.preprocessing import StandardScaler

# Create the scaler
ss = StandardScaler()

# Take a subset of the DataFrame you want to scale 
wine_subset = wine[['Ash', 'Alcalinity of ash', 'Magnesium']]

# Apply the scaler to the DataFrame subset
wine_subset_scaled = ss.fit_transform(wine_subset)

In [ ]:
# Import scale
from sklearn.preprocessing import scale

# Scale the features: X_scaled
X_scaled = scale(X)

# Print the mean and standard deviation of the unscaled features
print("Mean of Unscaled Features: {}".format(np.mean(X))) 
print("Standard Deviation of Unscaled Features: {}".format(np.std(X)))

# Print the mean and standard deviation of the scaled features
print("Mean of Scaled Features: {}".format(np.mean(X_scaled))) 
print("Standard Deviation of Scaled Features: {}".format(np.std(X_scaled)))

## Feature Engineering

### Numerical Features

In [ ]:
# Create a list of the columns to average
run_columns = ['run1', 'run2', 'run3', 'run4', 'run5']

# Use apply to create a mean column
running_times_5k["mean"] = running_times_5k.apply(lambda row: row[run_columns].mean(), axis=1)

# Take a look at the results
print(running_times_5k)

#### Datetime

In [ ]:
# Look at the first 5 rows of the date column
print(df_ufo.date.head())

# Extract the month from the date column
df_ufo["month"] = df_ufo["date"].apply(lambda d: d.month)

# Extract the year from the date column
df_ufo["year"] = df_ufo["date"].apply(lambda d: d.year)

# Print the head of all three columns
print(df_ufo[['date', 'month', 'year']].head())

### Categorical Features

When a variable/feature has two categories (or more than two categories and the categorical feature is ordinal) we can use Scikit-Learn's `LabelEncoder()` to encode it into 1s and 0s. When it has more than two categories and the feature is not ordinal, then we can perform one-hot encoding with pandas `get_dummies()` or Scikit-learn `OneHotEncoder()` functions.

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# Set up the LabelEncoder object
enc = LabelEncoder()

# Apply the encoding to the "Accessible" column
hiking['Accessible_enc'] = enc.fit_transform(hiking.Accessible)

# Compare the two columns
hiking[['Accessible_enc', 'Accessible']].head()

In [ ]:
# Transform the Region column of the df_gm
category_enc = pd.get_dummies(df_gm.Region)

#category_enc = pd.get_dummies(df_gm.Region, prefix_sep='_')

# Print the head of the encoded columns
category_enc.head()

#### Dealing with Multiple Categorical Variables

In [ ]:
categorize_label = lambda x: x.astype('category')

df.label = df[['label']].apply(categorize_label, axis=0)

### Text

In [ ]:
# Print the head of the length_of_time column
df_ufo.length_of_time.head()

In [ ]:
# import re

def return_minutes(time_string):

    # Use \d+ to grab digits
    pattern = re.compile(r"(\d+).min")

    # Use match on the pattern and column
    num = re.match(pattern, time_string)
    if num is not None:
        return int(num.group(1)) # return the first phranthesized subgroup
        
# Apply the extraction to the length_of_time column
df_ufo["minutes"] = df_ufo["length_of_time"].apply(lambda row: return_minutes(row))

# Print the head of both of the columns
df_ufo[['length_of_time', 'minutes']].head()

## Feature Selection

## Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the X and y sets using train_test_split, without stratify

X_train, X_test, y_train, y_test = train_test_split(X, y)

### Class Distribution

In [ ]:
# Split the X and y sets using train_test_split, without stratify

X_train, X_test, y_train, y_test = train_test_split(X, y)

If the distribution of a column is uneven, we may use **stratifying** in order to train the model on a sample which is representative of the dataset.

We want to predict which country the UFO sighting took place in. Therefore, we'll investigate the class distribution for this column.

In [ ]:
# Class distribution (or balance) for country column.
df_ufo["country"].value_counts()

In [ ]:
# Split the X and y sets using train_test_split, setting stratify=y
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)